# Pattern-4 (Dual-path / Skip-aware Fusion) — Kod Odaklı Anlatım

Bu notebook’un hedefi: **Pattern-4’ü sıfırdan kodlayıp**, bir attention modülüne **nasıl bağlandığını**, sonra da **normal bir modele nasıl entegre edildiğini** göstermek.

- Gereksiz teori yok.
- Kod akışı ve pratik tasarım adımları var.
- “Near-identity skip” mantığıyla stabil kurulum var.

---

## Pattern-4 özeti (1 satır)

\[ y = A_s(x) + A_r(F(x)) \]

- `A_s`: **skip** yolu için attention/gate (hafif, near-identity)
- `A_r`: **residual** yolu için attention/gate (daha güçlü olabilir)
- `F(·)`: residual dönüşüm (Conv/BN/ReLU/Conv/BN)



## 1) Pattern-4’ü kodlarken izlenecek yol (checklist)

1. **F(x)** dönüşümünü kur (BasicBlock benzeri)  
2. **Skip eşitleme** (stride/kanal değişiyorsa 1×1 conv) ekle  
3. Skip için **A_s** seç: genelde *hafif* (SE/ECA iyi gider)  
4. Residual için **A_r** seç: daha güçlü olabilir (SE/CBAM/CoordAtt)  
5. **Stabilite**: skip’i “tam kesen” maskeler yerine **near-identity** gate kullan  
   - `A_s(x) = x * (1 + γ_s * m_s(x))`  (γ_s küçük/0 başlar)  
6. Blok forward akışı:  
   - `identity = skip(x)`  
   - `s = A_s(identity)`  
   - `f = F(x)`  
   - `r = A_r(f)`  
   - `y = s + r` (+ opsiyonel ReLU)

Aşağıda bunu adım adım kuruyoruz.


## 2) Hazır parçalar: Conv-BN-ReLU ve skip eşitleme

In [1]:
import torch
import torch.nn as nn

def conv3x3(in_ch, out_ch, stride=1):
    return nn.Conv2d(in_ch, out_ch, kernel_size=3, stride=stride, padding=1, bias=False)

def conv1x1(in_ch, out_ch, stride=1):
    return nn.Conv2d(in_ch, out_ch, kernel_size=1, stride=stride, bias=False)

def make_skip(in_ch: int, out_ch: int, stride: int):
    if stride == 1 and in_ch == out_ch:
        return nn.Identity()
    return nn.Sequential(
        conv1x1(in_ch, out_ch, stride=stride),
        nn.BatchNorm2d(out_ch)
    ) 

## 3) Residual dönüşüm F(x) (BasicBlock tarzı)

Pattern-4’te F(x) “normal residual” ile aynıdır.  
Burada klasik tasarım:

- Conv → BN → ReLU
- Conv → BN
- (Toplama blokta yapılacak)


In [2]:
class FxConv(nn.Module):
    def __init__(self, in_ch: int, out_ch: int, stride: int = 1):
        super().__init__()
        self.conv1 = conv3x3(in_ch, out_ch, stride=stride)
        self.bn1   = nn.BatchNorm2d(out_ch)
        self.act   = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(out_ch, out_ch, stride=1)
        self.bn2   = nn.BatchNorm2d(out_ch)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        f = self.act(self.bn1(self.conv1(x)))
        f = self.bn2(self.conv2(f))
        return f

## 4) “Attention’a bağlamak” ne demek?

Pattern-4 için “attention’a bağlamak” şudur:

- Skip yolundaki tensöre bir **gating/attention** uygula → `A_s(·)`
- Residual yolundaki tensöre ayrı bir **gating/attention** uygula → `A_r(·)`

Önemli nokta: çoğu attention modülü direkt “çıktı” üretmez; bir **maske** üretir ve tensörü **çarpar**.

Bu notebookta:
- `SEMask`: kanal maskesi üretir (B,C,1,1)  
- `NearIdentityGate`: maskeyi `1 + γ*m` şeklinde uygular (skip güvenliği)


In [3]:
class SEMask(nn.Module):
    def __init__(self, channels: int, reduction: int = 16):
        super().__init__()
        hidden = max(channels // reduction, 4)
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.net = nn.Sequential(
            nn.Conv2d(channels, hidden, 1, bias=True),
            nn.ReLU(inplace=True),
            nn.Conv2d(hidden, channels, 1, bias=True),
            nn.Sigmoid(),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.net(self.pool(x))

class NearIdentityGate(nn.Module):
    def __init__(self, mask: nn.Module, gamma_init: float = 0.0, gamma_learnable: bool = True):
        super().__init__()
        self.mask = mask
        g = torch.tensor(float(gamma_init))
        if gamma_learnable:
            self.gamma = nn.Parameter(g)
        else:
            self.register_buffer("gamma", g)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        m = self.mask(x)          
        return x * (1.0 + self.gamma * m)

## 5) Pattern-4 blok: en temel hali (stabil)

Burada **iki gate** var:
- `A_s`: skip için (γ_s = 0 ile başlatılır → skip saf kalır)
- `A_r`: residual için (γ_r küçük pozitif başlayabilir)

Forward akışı:
1) identity = skip(x)  
2) s = A_s(identity)  
3) f = F(x)  
4) r = A_r(f)  
5) y = s + r  
6) ReLU


In [4]:
class Pattern4Block(nn.Module):
    def __init__(
        self,
        in_ch: int,
        out_ch: int,
        stride: int = 1,
        reduction_s: int = 16,
        reduction_r: int = 16,
        gamma_s_init: float = 0.0,   # skip: 0 ile başlat (near-identity)
        gamma_r_init: float = 0.1,   # residual: küçük pozitif olabilir
    ):
        super().__init__()
        self.skip = make_skip(in_ch, out_ch, stride)
        self.F = FxConv(in_ch, out_ch, stride=stride)

        # Skip gate (hafif)
        self.As = NearIdentityGate(SEMask(out_ch, reduction=reduction_s), gamma_init=gamma_s_init, gamma_learnable=True)

        # Residual gate (daha güçlü olabilir)
        self.Ar = NearIdentityGate(SEMask(out_ch, reduction=reduction_r), gamma_init=gamma_r_init, gamma_learnable=True)

        self.out_act = nn.ReLU(inplace=True)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        identity = self.skip(x)  # (B,out_ch,H',W')
        s = self.As(identity)

        f = self.F(x)            # (B,out_ch,H',W')
        r = self.Ar(f)

        y = s + r
        return self.out_act(y)
    
x = torch.randn(2, 64, 32, 32)
blk = Pattern4Block(64, 64, stride=1)
y = blk(x)
print('Pattern4Block out:', y.shape)
print('gamma_s:', float(blk.As.gamma), 'gamma_r:', float(blk.Ar.gamma))


Pattern4Block out: torch.Size([2, 64, 32, 32])
gamma_s: 0.0 gamma_r: 0.10000000149011612


## 6) “Daha ileri” kullanım: A_s ve A_r’yi farklı attention ile kurmak

Pattern-4’te A_s ve A_r **aynı olmak zorunda değil**.

Pratik kombinasyon:
- `A_s`: hafif (SE/ECA) → stabil
- `A_r`: daha güçlü (CBAM/CoordAtt) → ifade gücü

Bu notebook minimal kalsın diye SE kullandık.  
Ama Ar’yi CBAM ile değiştirmek istersen tek yapacağın şey:

- `self.Ar = NearIdentityGate(SEMask(out_ch), gamma_init=0.05~0.1)` yerine CBAM koymak.

Not: CBAM bazı implementasyonlarda “maskeyi” içeride uygular; o durumda `NearIdentityGate` yerine doğrudan `Ar(f)=CBAM(f)` kullanırsın.


## 7) Model entegrasyonu: “stage yok”, düz bir backbone

Pattern-4 bloklarını normal bir CNN akışına böyle sokarsın:

- stem
- conv
- Pattern4Block (same res)
- Pattern4Block (downsample)
- head

Aşağıdaki model, “tak-çalıştır” entegrasyon örneğidir.


In [5]:
class Pattern4Net(nn.Module):
    def __init__(self, num_classes: int = 10):
        super().__init__()

        self.stem = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
        )

        self.conv1 = nn.Sequential(
            nn.Conv2d(32, 64, 3, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
        )

        # Pattern-4 entegrasyonu
        self.p4_1 = Pattern4Block(64, 64, stride=1, gamma_s_init=0.0, gamma_r_init=0.1)
        self.p4_2 = Pattern4Block(64, 128, stride=2, gamma_s_init=0.0, gamma_r_init=0.1)

        self.conv2 = nn.Sequential(
            nn.Conv2d(128, 128, 3, padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
        )

        self.head = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(128, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.stem(x)
        x = self.conv1(x)
        x = self.p4_1(x)
        x = self.p4_2(x)
        x = self.conv2(x)
        x = self.head(x)
        return x

m = Pattern4Net(num_classes=10)
x = torch.randn(4, 3, 32, 32)
y = m(x)
print('Model out:', y.shape)


Model out: torch.Size([4, 10])


## 8) Pratik ayarlar (en çok iş gören)

- `gamma_s_init = 0.0`  → skip başlangıçta saf kalsın ✅  
- `gamma_r_init = 0.05–0.1` → residual gate hafif açık başlasın ✅  
- Pattern-4’ü **her yere basma**: orta/son kısımlar daha güvenli  
- Eğer eğitim dalgalanıyorsa:
  - LR düşür
  - gamma’ları küçük başlat (özellikle gamma_s)
  - Batch çok küçükse BN yerine GN düşün

Bu kadar. Pattern-4’ü “stabil ve kontrollü” kurmanın özü bu.
